In [ ]:
from keras.utils import np_utils
from tqdm import tqdm
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import matplotlib.pyplot as plt
import cv2
import os
import json
import csv
import collections
import pickle


<span style='background-color:#fff5b1'>data_dir는 사용자에 맞게 변경 필요</span>
------------------------------------------------

In [40]:
data_dir = 'C:/Users/kungm/Desktop/ETRI/example/'

Data preprocessing
=============

.json 파일 tranpose 후 다시 저장
--------------

In [36]:
train=[]
test=[]

with open(data_dir+"train.json", 'r',encoding='UTF8') as file:
    train = json.load(file)
    
with open(data_dir+"test.json", 'r',encoding='UTF8') as file:
    test = json.load(file)
train_df = pd.DataFrame(train)

In [41]:
len(train)

7683

In [38]:
#오디오 전처리에 사용하기 편하게 파일정리

train_df = pd.DataFrame(train)
train_df=train_df.transpose()
train_df=train_df.reset_index()
train_df=train_df.rename(columns={'index':'ID'})
train_df=train_df.drop(columns={'Text'})


test_df = pd.DataFrame(test)
test_df=test_df.transpose()
test_df=test_df.reset_index()
test_df=test_df.rename(columns={'index':'ID'})
test_df=test_df.drop(columns={'Text'})


In [39]:
train_df

,ID,Emotion,Label,WavPath
0,Sess01_script01_M029,surprise,1,./KEMDy19/wav/Session01/Sess01_script01/Sess01...
1,Sess01_script01_M012,fear,0,./KEMDy19/wav/Session01/Sess01_script01/Sess01...
2,Sess01_script06_M021,happy,5,./KEMDy19/wav/Session01/Sess01_script06/Sess01...
3,Sess01_script02_M024,neutral,4,./KEMDy19/wav/Session01/Sess01_script02/Sess01...
4,Sess01_script06_M009,disgust,6,./KEMDy19/wav/Session01/Sess01_script06/Sess01...
...,...,...,...,...
7678,Sess20_impro01_F020,neutral,4,./KEMDy19/wav/Session20/Sess20_impro01/Sess20_...
7679,Sess20_script06_F017,angry,2,./KEMDy19/wav/Session20/Sess20_script06/Sess20...
7680,Sess20_script06_F006,surprise,1,./KEMDy19/wav/Session20/Sess20_script06/Sess20...
7681,Sess20_impro04_F010,angry,2,./KEMDy19/wav/Session20/Sess20_impro04/Sess20_...


In [31]:
train_audio_tranpose=train_df.to_dict()
test_audio_tranpose=test_df.to_dict()

def save_json(dictionary, file_name):
    json_data = json.dumps(dictionary, ensure_ascii=False)
    with open(file_name, 'w', encoding='UTF8') as f:
        f.write(json_data)
        
save_json(train_audio_tranpose,  data_dir + 'train_audio.json')
save_json(test_audio_tranpose,  data_dir + 'test_audio.json')

In [32]:
#### json 파일 다시 불러와서 확인하는 부분 ####

train=[]
test=[]

with open(data_dir+'train_audio.json', 'r',encoding='UTF8') as file:
    train = json.load(file)
    
with open(data_dir+'test_audio.json', 'r',encoding='UTF8') as file:
    test = json.load(file)
    

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

In [33]:
train_df

,ID,Emotion,Label,WavPath
0,Sess01_script01_M029,surprise,1,./KEMDy19/wav/Session01/Sess01_script01/Sess01...
1,Sess01_script01_M012,fear,0,./KEMDy19/wav/Session01/Sess01_script01/Sess01...
2,Sess01_script06_M021,happy,5,./KEMDy19/wav/Session01/Sess01_script06/Sess01...
3,Sess01_script02_M024,neutral,4,./KEMDy19/wav/Session01/Sess01_script02/Sess01...
4,Sess01_script06_M009,disgust,6,./KEMDy19/wav/Session01/Sess01_script06/Sess01...
...,...,...,...,...
7678,Sess20_impro01_F020,neutral,4,./KEMDy19/wav/Session20/Sess20_impro01/Sess20_...
7679,Sess20_script06_F017,angry,2,./KEMDy19/wav/Session20/Sess20_script06/Sess20...
7680,Sess20_script06_F006,surprise,1,./KEMDy19/wav/Session20/Sess20_script06/Sess20...
7681,Sess20_impro04_F010,angry,2,./KEMDy19/wav/Session20/Sess20_impro04/Sess20_...


In [27]:
test_df

,ID,Emotion,Label,WavPath
0,Sess01_script01_M025,fear,0,./KEMDy19/wav/Session01/Sess01_script01/Sess01...
1,Sess01_impro02_M003,neutral,4,./KEMDy19/wav/Session01/Sess01_impro02/Sess01_...
2,Sess01_impro04_M028,happy,5,./KEMDy19/wav/Session01/Sess01_impro04/Sess01_...
3,Sess01_script02_M024,neutral,4,./KEMDy19/wav/Session01/Sess01_script02/Sess01...
4,Sess01_script06_M028,happy,5,./KEMDy19/wav/Session01/Sess01_script06/Sess01...
...,...,...,...,...
1938,Sess20_impro03_F007,surprise,1,./KEMDy19/wav/Session20/Sess20_impro03/Sess20_...
1939,Sess20_script01_F005,neutral,4,./KEMDy19/wav/Session20/Sess20_script01/Sess20...
1940,Sess20_impro02_F012,disgust,6,./KEMDy19/wav/Session20/Sess20_impro02/Sess20_...
1941,Sess20_impro02_F020,disgust,6,./KEMDy19/wav/Session20/Sess20_impro02/Sess20_...


Extraction feature
======================
<span style='background-color:#fff5b1'>custpath 사용자에 맞게 변경 필요,</span>
<span style='background-color:#fff5b1'>Wav 데이터 필요</span>
------------------------------------------------

 -Mel Spectrogram

In [ ]:
custpath = 'C:/Users/kungm/Desktop/ETRI/example/'#spectrogram 이미지 저장

def feature_extractor(row):

    name     = row[0]
    y, sr = librosa.load(row[3], sr=22050)
    try:
        
        audio = y
    
        #Spectrogram
        plt.axis('off') # no axis(선선)
        plt.axes([0., 0., 1., 1.])
        melspec  = librosa.feature.melspectrogram(y=audio)
        s_db     = librosa.power_to_db(melspec, ref=np.max) #소리 변환을 기록으로 인식할 수 있도록크기를 dB(데시벨)로 반전 크기를 전환합니다.
        librosa.display.specshow(s_db)
        savepath = os.path.join(custpath,name+'.png')
        plt.savefig(savepath, bbox_inches=None, pad_inches=0)
        plt.close()
        
    except:
        print('File cannot open:',name)
        return None
    return savepath

def resize_img(datafram):
    for i in tqdm(range(len(datafram))):
        file_path = datafram[i]
        img = cv2.imread(file_path)
        img = cv2.resize(img, (224, 224),interpolation = cv2.INTER_CUBIC)
        cv2.imwrite(file_path, img)

In [ ]:
imgpaths_train = []
imgpaths_test = []

idx = 0


for row in tqdm(train_df.values, leave=True):
    Feature_Set  = feature_extractor(row)
    imgpaths_train.append(Feature_Set)
    

for row in tqdm(test_df.values, leave=True):
    Feature_Set  = feature_extractor(row)
    imgpaths_test.append(Feature_Set)

#ResNet50 input 형식에 맞게 변경
resize_img(imgpaths_train)
resize_img(imgpaths_train)

Data generation
==============================

In [ ]:
data_train={}
data_train['ID']=[]
data_train['path']=[]
data_train['emotion']=[]

data_train_feature= [imgpaths_train[i] for i in range(len(imgpaths_train))]

for i in range(len(train_df)):
    data_train['ID'].append(train_df['ID'][i])
    data_train['path'].append(data_train_feature[i])
    data_train['emotion'].append(train_df['Label'][i])
    
data_test={}
data_test['ID']=[]
data_test['path']=[]
data_test['emotion']=[]

data_test_feature= [imgpaths_test[i] for i in range(len(imgpaths_test))]


for i in range(len(test_df)):
    data_test['ID'].append(test_df['ID'][i])
    data_test['path'].append(data_test_feature[i])
    data_test['emotion'].append(test_df['Label'][i])

In [ ]:

pickle.dump( data_train, open( data_dir + 'data_train.pickle', 'wb' ) )
pickle.dump( data_test, open( data_dir + 'data_test.pickle', 'wb' ) )

In [ ]:
#### data 파일 다시 불러와서 확인하는 부분 ####

data_train={}
data_train['ID']=[]
data_train['path']=[]
data_train['emotion']=[]

data_test={}
data_test['ID']=[]
data_test['path']=[]
data_test['emotion']=[]

with open( data_dir + 'data_train.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_train = pickle.load(f)
    


with open( data_dir + 'data_test.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    data_test = pickle.load(f)

        
ds_train = pd.DataFrame.from_dict(data_train)
ds_test = pd.DataFrame.from_dict(data_test)

In [ ]:
ds_train

In [ ]:
ds_test

Spectrogram이 없는 사용자도 모델을 학습시킬 수 있는 data 생성
==============================

In [ ]:
train={}
train['ID']=[]
train['path']=[]
train['emotion']=[]

for i in tqdm(range(len(data_train['path']))):
    seg_id= data_train['ID'][i]
    train['ID'].append(seg_id)
    img = cv2.imread(data_train['path'][i])
    train['path'].append(img)
    emotion = data_train['emotion'][i]
    train['emotion'].append(emotion)
    
    
test={}
test['ID']=[]
test['path']=[]
test['emotion']=[]

for i in tqdm(range(len(data_test['path']))):
    seg_id= data_test['ID'][i]
    test['ID'].append(seg_id)
    img = cv2.imread(data_test['path'][i])
    test['path'].append(img)
    emotion = data_test['emotion'][i]
    test['emotion'].append(emotion)
        

In [ ]:

pickle.dump( train, open( data_dir + 'train_final.pickle', 'wb' ) )
pickle.dump( test, open( data_dir + 'test_final.pickle', 'wb' ) )

In [ ]:
train={}
train['ID']=[]
train['path']=[]
train['emotion']=[]
    
test={}
test['ID']=[]
test['path']=[]
test['emotion']=[]

with open( data_dir + 'train_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    train = pickle.load(f)
    


with open( data_dir + 'test_final.pickle', 'rb') as f:
    #파일을 열고 닫는 것을 자동으로 처리
    test = pickle.load(f)

        
ds_train = pd.DataFrame.from_dict(train)
ds_test = pd.DataFrame.from_dict(test)

In [ ]:
ds_test

In [ ]:
ds_train